In [ ]:
# Import Training and Testing Data
import pandas as pd

online_booking_train = pd.read_csv("train-online_booking_2020.csv")
online_booking_test = pd.read_csv("test-online_booking_2021.csv")

In [ ]:
# Checking Missing Value from the Data
df1 = online_booking_train.isna().sum()
df2 = online_booking_test.isna().sum()
print("Data testing:\n", df1, "\n")
print("Data training:\n", df2)

In [ ]:
# Making the Used Room Dataset
online_booking_train['room_used'] = online_booking_train['room_total'] - online_booking_train['all_available_room']
online_booking_test['room_used'] = online_booking_test['room_total'] - online_booking_test['all_available_room']

In [ ]:
online_booking_train['tpk_online'] = online_booking_train['room_used']/online_booking_train['room_total']*100
online_booking_test['tpk_online'] = online_booking_test['room_used']/online_booking_test['room_total']*100

In [ ]:
# Convert the Type of 'tanggal' Column into Datetime Type
online_booking_train['tanggal'] = pd.to_datetime(online_booking_train['tanggal'])
online_booking_test['tanggal'] = pd.to_datetime(online_booking_test['tanggal'])

In [ ]:
print(online_booking_train.head())
print(online_booking_test.head())

In [ ]:
# Visualize the Train Data Depend On the Time Aggregate
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
online_booking_train.groupby('tanggal')['room_used'].sum().plot()
plt.title("Jumlah Ruang yang Digunakan Berdasarkan Waktu", fontsize=15)
plt.xlabel("Waktu")
plt.ylabel("Jumlah")
plt.show()

In [ ]:
# Setting the Datetime Dataset ('tanggal') Column to Only Group of Month
agg_train_online_booking = online_booking_train.set_index('tanggal').resample('M').mean()
agg_test_online_booking = online_booking_test.set_index('tanggal').resample('M').mean()

In [ ]:
# Reset the Data Index
agg_train_online_booking = agg_train_online_booking.reset_index()
agg_test_online_booking = agg_test_online_booking.reset_index()

In [ ]:
# Set the ID depend on Time Column
agg_train_online_booking['Id'] = pd.DatetimeIndex(agg_train_online_booking['tanggal']).month
agg_test_online_booking['Id'] = pd.DatetimeIndex(agg_test_online_booking['tanggal']).month

In [ ]:
# Import TPK Dataset
tpk_hotel_berbintang_train = pd.read_csv("train-TPK_Hotel_berbintang_2020.csv")
tpk_hotel_berbintang_test = pd.read_csv("test-TPK_Hotel_berbintang_2021.csv")

In [ ]:
# Merge All of The Data into a Completed Training and Testing Dataset
df_full_train = pd.merge(agg_train_online_booking, tpk_hotel_berbintang_train, on='Id', how='left')
df_full_test = pd.merge(agg_test_online_booking, tpk_hotel_berbintang_test, on='Id', how='left')

In [ ]:
print(df_full_train.head())
print(df_full_test.head())

In [ ]:
# Fill the Aggregate Dataset using The Mean of the Used Room
agg_value_train = df_full_train.groupby('tanggal')['room_used'].mean()
agg_value_test = df_full_test.groupby('tanggal')['room_used'].mean()

for x in range(0, 12):
    df_full_train.loc[:x, 'Aggregate_var'].fillna(agg_value_train[x], inplace=True)

for y in range(0, 6):
    df_full_test.loc[:y, 'Aggregate'].fillna(agg_value_test[y], inplace=True)

In [ ]:
# Making Training and Testing Dataset
train_ds = df_full_train[['tpk_online','TPK']]
test_ds = df_full_test[['tpk_online', 'TPK']]

In [ ]:
# Checking the Normalization of Data
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
sns.displot(test_ds, x='tpk_online_minmaxscaler')
plt.show()

In [ ]:
# Checking The Outlier
plt.figure(figsize=(10,8))
train_ds.boxplot()
plt.show()

In [ ]:
# Second Splitting
X_train_ds = train_ds['tpk_online']
y_train_ds = train_ds['TPK']

In [ ]:
# Making the Decision Tree Regressor Model
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import numpy as np

dtr = DecisionTreeRegressor()
dtr.get_params()

In [ ]:
grid = GridSearchCV(dtr, param_grid=dict(
                                        ccp_alpha=np.linspace(1, 10, 10),
                                        criterion=['mse','mae']
))
model = grid.fit(X_train_ds.values.reshape(-1,1), y_train_ds.values.reshape(-1,1))

y_pred = model.predict(X_train_ds.values.reshape(-1,1))
print("Model score:", model.score(X_train_ds.values.reshape(-1,1), y_train_ds.values.reshape(-1,1)))

In [ ]:
# Making the Decision Tree Regressor Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

rfc = RandomForestRegressor()
rfc.get_params()

In [ ]:
grid = GridSearchCV(rfc, param_grid=dict(
                                        ccp_alpha=np.linspace(1, 10, 10),
                                        criterion=['mse', 'mae'],
), n_jobs=-1)
model = grid.fit(X_train_ds.values.reshape(-1,1), y_train_ds.values.reshape(-1,1))

y_pred = model.predict(X_train_ds.values.reshape(-1,1))
print("Model score:", model.score(X_train_ds.values.reshape(-1,1), y_train_ds.values.reshape(-1,1)))

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.get_params()

In [ ]:
grid = GridSearchCV(ridge, param_grid=dict(
                                            alpha=(2.5, 3, 10, 11),
                                            normalize=['True', 'False']),
                                            n_jobs=-1
)

model = grid.fit(X_train_ds.values.reshape(-1,1), y_train_ds.values.reshape(-1,1))

y_pred = model.predict(X_train_ds.values.reshape(-1,1))
print("Model score:", model.score(X_train_ds.values.reshape(-1,1), y_train_ds.values.reshape(-1,1)))

In [ ]:
# Check the Root Mean Squared Error of the Model
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(X_training, y_pred)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
# Predict the Testing TPK Dataset
tpk_predict = model.predict(test_ds['tpk_online'].values.reshape(-1,1))
tpk_predict = tpk_predict.ravel()
tpk_predict

In [ ]:
# Fill the Resul of the TPK Prediction
for y in range(0, 6):
    df_full_test.loc[:y, 'TPK'].fillna(tpk_predict[y], inplace=True)

In [ ]:
df_full_test['TPK']

In [64]:
hasil = pd.DataFrame()
hasil['Id'] = df_full_test['Id']
hasil['TPK'] = df_full_test['TPK']
hasil[['Id', 'TPK']]

,Id,TPK
0,1,11.705102
1,2,13.010614
2,3,11.463698
3,4,12.229685
4,5,12.383181
5,6,12.970386


In [65]:
hasil.to_csv('HASIL_MODEL.csv', index=False)

In [ ]:
# Checking the Confusion Matrix of the Model
from yellowbrick.regressor import PredictionError

pe = PredictionError(dtr)
pe.fit(X_train.values.reshape(-1,1), y_train.values.reshape(-1,1))
pe.score(X_test.values.reshape(-1,1), y_test.values.reshape(-1,1))
pe.show()